#### Imports

In [251]:
import numpy as np
from PIL import ImageGrab #grabbing image
from PIL import Image
import cv2 #opencv
import io
import time
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (30, 30)
import seaborn as sns
import pandas as pd
import numpy as np
from random import randint
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
#keras imports
%matplotlib inline 
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
import json

In [252]:
#path variables
game_url = "game/snake.html"
chrome_driver_path = "../chromedriver.exe"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
scores_file_path = "./objects/scores_df.csv"

### Game Module
This is the main module that implements interfacing between the python and browser-javascript using selenium


In [253]:
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        self._driver = webdriver.Chrome(executable_path = "chromedriver.exe",chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(400,500)
        self._driver.get(os.path.abspath(game_url))
        #modifying game before trainNetworkining
#         if custom_config:
#             self._driver.execute_script("Runner.config.ACCELERATION=0")
    def get_crashed(self):
        return self._driver.execute_script("return Init.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Init.instance_.playing")
    def restart(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ENTER)
        
        time.sleep(0.25)# no actions are possible 
                        # for 0.25 sec after game starts, 
                        # skip learning at this time and make the model wait
    def press_enter(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ENTER)
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def press_down(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_DOWN)
    def press_left(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_LEFT)
    def press_right(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_RIGHT)
        
    def get_score(self):
#         score_array = self._driver.execute_script("return Init.instance_.score")
        score = self._driver.execute_script("return Init.instance_.score")
        return int(score)
#     def pause(self):
#         return self._driver.execute_script("return Runner.instance_.stop()")
#     def resume(self):
#         return self._driver.execute_script("return Runner.instance_.play()")
    def end(self):
        self._driver.close()

### Agent module
This model represent the agent (Dino) which the model controls for playing. 

In [254]:
class SnakeAgent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game; 
        self.start(); #to start the game, we need to jump once
        time.sleep(.5) # no action can be performed for the first time when game starts
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def up(self):
        self._game.press_up()
    def start(self):
        self._game.press_enter()
    def down(self):
        self._game.press_down()
    def left(self):
        self._game.press_left()
    def right(self):
        self._game.press_right()

In [255]:
# Game()

### Game state module
Game state helps to get the current state of the game environment as well as the agent.<br>
Actions are performed by this model before getting a new state

In [256]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
#         actions_df.loc[len(actions_df)] = actions[1:]
        print(actions[1:])# storing actions in a dataframe
        score = self._game.get_score() 
        reward = 0.01*score - 0.01 # dynamic reward calculation
        is_over = False #game over
        if actions[1] == 1:
            self._agent.left()
#             actions = [0,-1,-1,0,0]
            reward = 0.1*score
        elif actions[2] == 1:
            self._agent.right()
#             actions = [0,-1,-1,0,0]
            reward = 0.1*score
        elif actions[3] == 1:
            self._agent.up()
#             actions = [0,0,0,-1,-1]
            reward = 0.1*score
        elif actions[4] == 1:
            self._agent.down()
#             actions = [0,0,0,-1,-1]
            reward = 0.1*score
        image = grab_screen() 
        self._display.send(image) #display the image on screen

        if self._agent.is_crashed():
#             scores_df.loc[len(loss_df)] = score # log the score when game is over
            self._game.restart()
            if score == 0:
                reward = 0.0
            else:
                reward = score
            is_over = True
        return image, reward, is_over #return the Experience tuple

In [257]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
def grab_screen(_driver = None):
    screen =  np.array(ImageGrab.grab(bbox=(0,210,380,430))) #bbox = region of interset on the entire screen
    image = process_img(screen)#processing image as required
    return image

def process_img(image):
    #game is already in grey scale canvas, canny to get only edges and reduce unwanted objects(clouds)
    image = cv2.resize(image, (0,0), fx = 0.4, fy = 0.4) # resale image dimensions
#     image = image[2:38,10:50] #img[y:y+h, x:x+w] #crop out the dino agent from the frame
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200) #apply the canny edge detection
    return  image
def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [258]:
# Game()

In [259]:
# # g = Game()
# # s = SnakeAgent(g)
# # Game_sate(s,g)
# i = grab_screen()
# i = process_img(i)
# i = cv2.resize(i, (800, 400)) 
# cv2.imshow('window_title', i)
# ImageGrab.grab(bbox=(0,210,380,430))
# m = np.array(ImageGrab.grab(bbox=(0,210,380,430)))
# m  = cv2.resize(m, (0,0), fx = 0.4, fy = 0.4)
# m = cv2.Canny(m, threshold1 = 100, threshold2 = 200)
# m = cv2.resize(m, (800, 400))
# cv2.imwrite('color_img.jpg', m)
# cv2.imshow("image", m);
# cv2.waitKey();

### Intialize log structures

In [260]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])

In [261]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

#### Module parameters

In [262]:
#game parameters
ACTIONS = 5 # possible actions
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 1000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 8
LEARNING_RATE = 1e-4
img_rows , img_cols = 152,88
img_channels = 4 #We stack 4 frames

In [263]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same',input_shape=(img_cols,img_rows,img_channels)))  #20*40*4
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

### Mode Architecture

In [264]:
buildmodel().summary()

Now we build the model
We finish building the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_106 (Conv2D)          (None, 22, 38, 32)        8224      
_________________________________________________________________
activation_141 (Activation)  (None, 22, 38, 32)        0         
_________________________________________________________________
conv2d_107 (Conv2D)          (None, 11, 19, 64)        32832     
_________________________________________________________________
activation_142 (Activation)  (None, 11, 19, 64)        0         
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 11, 19, 64)        36928     
_________________________________________________________________
activation_143 (Activation)  (None, 11, 19, 64)        0         
_________________________________________________________________
flatten_36 (Flatten)    

In [265]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = deque() #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1 #0 =>do nothing, 1=>left, 2=>right, 3=>up, 4=>down
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
#     acts = [0,-1,-1,0,0]

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
#         model.load_weights("model_final.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON
#         model.load_weights("model_final.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
    t=0
#     t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
#                 while True:
#                     action_index = random.randrange(ACTIONS)
#                     if a_t[action_index] != -1
#                         a_t[action_index] = 1
#                         break
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # 0=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('reward: {}'.format(r_t))
        print('loop took {} seconds'.format(time.time()-last_time)) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
        else:
            # artificial time delay as training done with this delay
            time.sleep(0.12)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0 and False:
            print("Now we save model")
            
            model.save_weights("model_final.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [266]:
def playGame(observe=False):
    game = Game()
    dino = SnakeAgent(game)
    game_state = Game_sate(dino,game)
    model = buildmodel()
    try:
        trainNetwork(model,game_state)
    except StopIteration:
        game.end()

In [267]:
# init_cache()

In [268]:
playGame(observe=False);

c:\users\chouh\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


Now we build the model
We finish building the model
[0. 0. 0. 0.]
[0. 0. 0. 1.]
reward: 0.0
loop took 3.2390940189361572 seconds
TIMESTEP 1 / STATE observe / EPSILON 0.1 / ACTION 4 / REWARD 0.0 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.2630763053894043 seconds
TIMESTEP 2 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.18396711349487305 seconds
TIMESTEP 3 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.21504998207092285 seconds
TIMESTEP 4 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.23902153968811035 seconds
TIMESTEP 5 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.17717695236206055 seconds
TIMESTEP 6 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
re

reward: -0.01
loop took 0.24536371231079102 seconds
TIMESTEP 54 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.25061964988708496 seconds
TIMESTEP 55 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.1999671459197998 seconds
TIMESTEP 56 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 1.]
reward: 0.0
loop took 0.3197824954986572 seconds
TIMESTEP 57 / STATE observe / EPSILON 0.1 / ACTION 4 / REWARD 0.0 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.24071216583251953 seconds
TIMESTEP 58 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.24592089653015137 seconds
TIMESTEP 59 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -0.01 / Q_MAX  0 / Loss  0
[0. 0. 0. 0.]
reward: -0.01
loop took 0.24397587776184082 seconds
TIMESTEP 60 / STATE obse

TIMESTEP 107 / STATE explore / EPSILON 0.0994006 / ACTION 0 / REWARD -0.01 / Q_MAX  110.135345 / Loss  213.67239379882812
[0. 0. 0. 0.]
reward: -0.01
loop took 0.754805326461792 seconds
TIMESTEP 108 / STATE explore / EPSILON 0.0993007 / ACTION 0 / REWARD -0.01 / Q_MAX  127.94391 / Loss  96.79328918457031
[0. 0. 0. 0.]
reward: 0.0
loop took 1.1099917888641357 seconds
TIMESTEP 109 / STATE explore / EPSILON 0.0992008 / ACTION 0 / REWARD 0.0 / Q_MAX  138.44994 / Loss  390.1247253417969
[0. 0. 0. 0.]
reward: -0.01
loop took 1.3166999816894531 seconds
TIMESTEP 110 / STATE explore / EPSILON 0.0991009 / ACTION 0 / REWARD -0.01 / Q_MAX  151.07494 / Loss  335.8787841796875
[0. 0. 0. 0.]
reward: -0.01
loop took 0.7050845623016357 seconds
TIMESTEP 111 / STATE explore / EPSILON 0.099001 / ACTION 0 / REWARD -0.01 / Q_MAX  143.57977 / Loss  334.96875
[0. 0. 0. 0.]
reward: -0.01
loop took 0.6941468715667725 seconds
TIMESTEP 112 / STATE explore / EPSILON 0.0989011 / ACTION 0 / REWARD -0.01 / Q_MAX  146

TIMESTEP 152 / STATE explore / EPSILON 0.0949051 / ACTION 0 / REWARD -0.01 / Q_MAX  144.40791 / Loss  187.00485229492188
[0. 0. 0. 1.]
reward: 0.0
loop took 1.1160845756530762 seconds
TIMESTEP 153 / STATE explore / EPSILON 0.0948052 / ACTION 4 / REWARD 0.0 / Q_MAX  156.53279 / Loss  144.64222717285156
[0. 0. 0. 0.]
reward: -0.01
loop took 0.9392538070678711 seconds
TIMESTEP 154 / STATE explore / EPSILON 0.0947053 / ACTION 0 / REWARD -0.01 / Q_MAX  128.01764 / Loss  197.25064086914062
[0. 0. 0. 0.]
reward: -0.01
loop took 1.0706021785736084 seconds
TIMESTEP 155 / STATE explore / EPSILON 0.0946054 / ACTION 0 / REWARD -0.01 / Q_MAX  123.823975 / Loss  150.58839416503906
[0. 0. 0. 0.]
reward: 0.0
loop took 1.3297865390777588 seconds
TIMESTEP 156 / STATE explore / EPSILON 0.0945055 / ACTION 0 / REWARD 0.0 / Q_MAX  139.88803 / Loss  232.56521606445312
[0. 0. 0. 0.]
reward: -0.01
loop took 1.1282250881195068 seconds
TIMESTEP 157 / STATE explore / EPSILON 0.0944056 / ACTION 0 / REWARD -0.01 / 

...

TIMESTEP 456 / STATE explore / EPSILON 0.06453550000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  120.56356 / Loss  43.796302795410156
[0. 0. 0. 0.]
reward: -0.01
loop took 0.8919847011566162 seconds
TIMESTEP 457 / STATE explore / EPSILON 0.06443560000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  80.396065 / Loss  70.43374633789062
[0. 0. 0. 0.]
reward: 0.0
loop took 1.132598876953125 seconds
TIMESTEP 458 / STATE explore / EPSILON 0.06433570000000001 / ACTION 0 / REWARD 0.0 / Q_MAX  97.17613 / Loss  45.109779357910156
[0. 0. 0. 0.]
reward: -0.01
loop took 1.011171579360962 seconds
TIMESTEP 459 / STATE explore / EPSILON 0.06423580000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  71.61705 / Loss  71.28620147705078
[0. 0. 0. 0.]
reward: -0.01
loop took 0.695915699005127 seconds
TIMESTEP 460 / STATE explore / EPSILON 0.06413590000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  87.880554 / Loss  68.87821197509766
[0. 0. 0. 0.]
reward: -0.01
loop took 0.7532808780670166 seconds
TIMESTEP 461 / STATE explore / E

TIMESTEP 499 / STATE explore / EPSILON 0.06023980000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  69.59752 / Loss  8.697717666625977
[0. 0. 0. 0.]
reward: 0.0
loop took 1.1422982215881348 seconds
TIMESTEP 500 / STATE explore / EPSILON 0.06013990000000001 / ACTION 0 / REWARD 0.0 / Q_MAX  69.07401 / Loss  4.726713180541992
[0. 0. 0. 0.]
reward: -0.01
loop took 1.1389884948730469 seconds
TIMESTEP 501 / STATE explore / EPSILON 0.06004000000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  88.067986 / Loss  5.651064872741699
[0. 0. 0. 0.]
reward: -0.01
loop took 0.7457442283630371 seconds
TIMESTEP 502 / STATE explore / EPSILON 0.05994010000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  81.09053 / Loss  7.2344584465026855
[0. 0. 0. 0.]
reward: -0.01
loop took 0.7124614715576172 seconds
TIMESTEP 503 / STATE explore / EPSILON 0.05984020000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  68.72039 / Loss  6.013083457946777
[0. 0. 0. 0.]
reward: -0.01
loop took 0.7047016620635986 seconds
TIMESTEP 504 / STATE explore / E

TIMESTEP 542 / STATE explore / EPSILON 0.05594410000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  73.753075 / Loss  6.95079231262207
[0. 0. 0. 0.]
reward: 0.0
loop took 1.0210976600646973 seconds
TIMESTEP 543 / STATE explore / EPSILON 0.05584420000000001 / ACTION 0 / REWARD 0.0 / Q_MAX  82.69623 / Loss  6.573667049407959
[0. 0. 0. 0.]
reward: -0.01
loop took 1.0026016235351562 seconds
TIMESTEP 544 / STATE explore / EPSILON 0.05574430000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  74.82821 / Loss  11.25205135345459
[0. 0. 0. 0.]
reward: -0.01
loop took 0.734748125076294 seconds
TIMESTEP 545 / STATE explore / EPSILON 0.05564440000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  80.589966 / Loss  12.779617309570312
[0. 0. 0. 0.]
reward: -0.01
loop took 0.6892907619476318 seconds
TIMESTEP 546 / STATE explore / EPSILON 0.05554450000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  87.87698 / Loss  10.451245307922363
[0. 0. 0. 0.]
reward: -0.01
loop took 0.6824514865875244 seconds
TIMESTEP 547 / STATE explore / E

TIMESTEP 585 / STATE explore / EPSILON 0.05164840000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  77.09392 / Loss  86.25477600097656
[0. 0. 0. 0.]
reward: -0.01
loop took 1.1133348941802979 seconds
TIMESTEP 586 / STATE explore / EPSILON 0.05154850000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  71.68775 / Loss  40.1387825012207
[0. 0. 0. 0.]
reward: -0.01
loop took 1.082216501235962 seconds
TIMESTEP 587 / STATE explore / EPSILON 0.05144860000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  83.31858 / Loss  115.73211669921875
[0. 0. 0. 0.]
reward: -0.01
loop took 1.0879371166229248 seconds
TIMESTEP 588 / STATE explore / EPSILON 0.05134870000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  80.707184 / Loss  80.96762084960938
[0. 0. 0. 0.]
reward: -0.01
loop took 1.152000904083252 seconds
TIMESTEP 589 / STATE explore / EPSILON 0.05124880000000001 / ACTION 0 / REWARD -0.01 / Q_MAX  77.52226 / Loss  70.29627227783203
[0. 0. 0. 0.]
reward: -0.01
loop took 1.0360956192016602 seconds
TIMESTEP 590 / STATE explore / 

KeyboardInterrupt: 

### Results
<strong>Loss and Scores :</strong> I trained my model for around 2 million frames for a week. 1st million steps were used for fine tuning the game parameters and fixing bugs.Plotting the last million loss values and all scores record for all the game.  The last million training frames showed improvement in game scores reaching a maximum score of 265 till now. We can observe that loss has stabilized for the last million steps and stays low with minute fluctuations.

<strong>Action Distribution</strong>:
The density distribution of the actions performed by the model considerd for evaulation. 


##### Plotting the game training logs

### import matplotlib.animation as animation


fig, axs = plt.subplots(ncols=1,nrows =2,figsize=(15,15))
def show_plots(i):
    axs[0].set_title('Loss')
    axs[1].set_title('Game Score progress')
    loss_df = pd.read_csv("./objects/loss_df.csv").clip(0,50).tail(100000)
    scores_df = pd.read_csv("./objects/scores_df.csv").head(190000)
    
    actions_df = pd.read_csv("./objects/actions_df.csv").tail(100000)
    loss_df['loss'] = loss_df['loss'].astype('float') 
    loss_df.plot(use_index=True,ax=axs[0])
    scores_df.plot(ax=axs[1])
#     sns.distplot(actions_df,ax=axs[2])
    imgg = fig.canvas.draw()

animate = animation.FuncAnimation(fig, show_plots, interval=2000)

### Comparision between human and AI
A gameplay of human was recorded for a score of 500.
The plot below shows the distrubtion comparision of actions performed
It can be inferred from the comparision that the Model has currently adopted a policy that favours jumps more as compared to doing nothing and which is the main reason the dino can be seen to crash. This behaviour is a sign of noise in the training phase for which more exploration(with higher bias towards 'do nothing' action) training is required.

#### Ploting and comparing AI and human gameplay action distribution

In [ ]:
#training_data_final_working.npy file contains the the keystrokes and gameframes recording for a score of 500
supervised_frames = np.load("training_data_final_working.npy")
frame = supervised_frames[0][0]
action_index = supervised_frames[0][1]
#plotting a sample frame from human recorded gameplay
plt.imshow(frame)
print('Action taken at this frame : Action index = {} i.e. jump'.format(str(action_index)))

In [ ]:
supervised_actions = []

for frame in supervised_frames:
    supervised_actions.append(frame[1])

In [ ]:
fig, axs = plt.subplots(ncols=1,nrows =2,figsize=(15,15))
sns.distplot(supervised_actions,ax=axs[0])
axs[1].set_title('AI gameplay distribution')
axs[0].set_title('Human gameplay distribution')
actions_df = pd.read_csv("./objects/actions_df.csv")
sns.distplot(actions_df,ax=axs[1])

### Credits
1. Volodymyr Mnih, Koray Kavukcuoglu, David Silver, Alex Graves, Ioannis Antonoglou,
Daan Wierstra,  and Martin Riedmiller. ‘Playing Atari with Deep Reinforcement Learning’ arXiv:1312.5602, 19 Dec 2013
2. Kevin Chen, Deep Reinforcement Learning for Flappy Bird.
3. Gerald Tesauro. Temporal difference learning and td-gammon. Communications of the ACM, 38(3):58–68, 1995.
4. Toy example of a deep reinforcement learning model playing a game of catching fruit, https://github.com/bitwise-ben/Fruit
5. MNIH, Volodymyr, et al. Human-level control through deep reinforcement learning. Nature, 2015, vol. 518, no 7540, p. 529-533.
6. Tambet Matiisen. Demystifying Deep Reinforcement Learning https://ai.intel.com/demystifying-deep-reinforcement-learning/
7. Sascha Lange, Thomas Gabel, and Martin Riedmiller. Batch Reinforcement Learning
8. Using Deep Q-Network to Learn How To Play Flappy Bird   https://github.com/yenchenlin/DeepLearningFlappyBird
9. The image processing modules were inspired from Harrison Sentdex's github which is licensed under the GNU GENERAL PUBLIC LICENSE http://www.gnu.org/licenses/gpl.html<br>
10. Coroutine implementation in the code below belongs to Ben Meijering and is licensed under the MIT License https://opensource.org/licenses/MIT